In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_name = 'spvae'

data_file = f'../{data_name}_vectors_drags_lifts.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128,drag,lift
406,-0.000353,0.938900,0.730082,0.001080,-0.723818,-0.000713,-0.363452,1.363965,0.000757,1.158052,...,0.000228,0.000479,0.000233,-0.000070,-1.169997,-0.630202,-0.001056,0.000770,0.395,0.425
54,0.000310,-0.932797,0.025703,0.000571,-1.087611,0.000181,0.035749,-0.290552,-0.030080,0.761045,...,0.000569,-0.001818,-0.000403,0.000658,0.668129,2.250648,-0.010414,0.001804,0.645,0.885
241,-0.000317,-1.111809,-1.151887,-0.000197,-0.180751,0.000557,-0.561294,0.094284,0.002247,0.794197,...,0.000313,-0.000366,-0.000735,0.001392,-0.409571,-2.031675,-0.001192,0.000625,0.446,0.756
952,0.000499,0.457167,-1.479584,0.001333,-0.577155,0.000301,-0.905196,-1.337308,-0.001215,-1.755122,...,0.000282,-0.000194,-0.000080,-0.000260,-0.697985,-1.125502,-0.000115,0.000077,0.384,0.177
337,0.000667,0.547938,0.491766,0.000550,0.434486,0.000730,3.182191,-0.781295,-0.002309,1.794750,...,0.000391,-0.000374,-0.000856,-0.000261,0.332565,0.792273,-0.000045,-0.000629,0.381,0.074


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    837.000000
mean       0.445233
std        0.112500
min        0.051000
25%        0.369000
50%        0.427000
75%        0.513000
max        0.944000
Name: drag, dtype: float64


In [3]:
import os
save_path = f'./agModels-{data_name}_{label}'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-spvae_drag/learner.pkl
Saving ./agModels-spvae_drag/predictor.pkl

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128,lift
1046,0.000482,-1.632118,0.521768,0.000057,-0.524977,0.001181,-1.053361,-0.710915,-0.014259,-0.247215,...,0.940937,-0.000201,-0.000543,-0.000332,-0.000086,-0.250152,0.810739,-0.005433,0.000972,0.346
657,0.000334,-0.717807,0.156846,0.000663,-0.597166,0.000386,-2.051708,0.732814,-0.017638,-0.092468,...,-1.009466,-0.000286,-0.001223,-0.000568,0.000159,0.285532,0.058386,-0.006653,0.000682,0.654
127,-0.001045,0.985687,0.333722,-0.000779,0.795979,0.000015,0.222498,1.193969,-0.002246,0.801656,...,1.449672,0.000892,-0.000499,-0.000412,0.001091,0.325954,0.986142,-0.002773,0.000732,0.052
951,-0.000010,-1.110299,-1.179036,0.001446,0.173944,0.000330,0.236702,-0.755380,-0.002542,-1.876672,...,-2.466808,0.000105,0.000189,0.001414,-0.000600,0.408066,1.761171,-0.000150,0.001450,0.604
134,-0.000103,2.300475,-0.223271,0.001451,0.734430,0.000361,0.682707,-0.307900,-0.000511,2.187333,...,-0.736073,-0.000425,0.000223,0.000188,-0.001001,-1.714110,-0.823680,-0.000549,0.000145,0.199


In [5]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_all_parts.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-spvae_drag/predictor.pkl
Loading: ./agModels-spvae_drag/learner.pkl
Loading: ./agModels-spvae_drag/models/trainer.pkl
Loading: ./agModels-spvae_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/WeightedEnsemble_L2/model.pkl
Evaluation: root_mean_squared_error on test data: -0.08675923795988598
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.08675923795988598,
    "mean_squared_error": -0.00752716537138012,
    "mean_absolute_error": -0.06141823123750233,
    "r2": 0.41180375983855055,
    "pearsonr": 0.6424676394482015,
    "median_absolute_error": -0.0453535534143448
}
Loading: ./agModels-spvae_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/model

0.45553457736968994
0.349494606256485
0.4757624864578247
0.44975313544273376
0.48073655366897583
0.45665115118026733
0.3808498978614807
0.3737154006958008
0.34817999601364136
0.4167404770851135
0.5027877688407898
0.4395475685596466
0.4065016508102417
0.3692750632762909
0.6026980876922607
0.5472506284713745
0.34648603200912476
0.512423038482666
0.5288976430892944
0.4139365255832672
0.45063316822052
0.407410591840744
0.576930046081543
0.40010055899620056
0.3738037645816803
0.39068999886512756
0.5814734697341919
0.3807024359703064
0.48539015650749207
0.3484039604663849
0.41499969363212585
0.4621046781539917
0.43766775727272034
0.568122923374176
0.4383101463317871
0.40096908807754517
0.41513627767562866
0.4234981834888458
0.34891974925994873
0.45731791853904724
0.45669856667518616
0.35035204887390137
0.4790354371070862
0.4342949390411377
0.5271941423416138
0.37207379937171936
0.42278751730918884
0.46201077103614807
0.3989633023738861
0.35322731733322144
0.4550298750400543
0.431362926959991

Loading: ./agModels-spvae_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-spvae_drag/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-spvae_drag/models/LightGBM_BAG_L2/model.pkl
Loading: ./agModels-spvae_drag/models/RandomForestMSE_B

                     model  score_test  score_val  pred_time_test  \
0          CatBoost_BAG_L2   -0.084393  -0.084315       15.134253   
1        LightGBMXT_BAG_L2   -0.084619  -0.083874       16.221649   
2      WeightedEnsemble_L3   -0.084654  -0.083160       19.586156   
3      WeightedEnsemble_L4   -0.084757  -0.085114       32.167030   
4    NeuralNetTorch_BAG_L3   -0.084782  -0.086183       30.146612   
5          CatBoost_BAG_L3   -0.084889  -0.085398       28.492121   
6   RandomForestMSE_BAG_L2   -0.084895  -0.083896       15.208333   
7     ExtraTreesMSE_BAG_L2   -0.085035  -0.083634       15.214768   
8           XGBoost_BAG_L2   -0.085110  -0.084926       15.420329   
9    NeuralNetTorch_BAG_L4   -0.085334  -0.088772       39.925006   
10         LightGBM_BAG_L2   -0.085356  -0.084364       15.799162   
11       LightGBMXT_BAG_L3   -0.085653  -0.086850       29.636896   
12   NeuralNetTorch_BAG_L2   -0.085916  -0.086490       19.079224   
13    LightGBMLarge_BAG_L2   -0.08

In [6]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 129 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [7]:
train_data_pred = predictor.predict(train_data)
test_data_pred = predictor.predict(test_data)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt(f'./{data_name}_vectors_y_test_hat_{label}.csv', test_data_pred, delimiter=',')
np.savetxt(f'./{data_name}_vectors_y_train_hat_{label}.csv', train_data_pred, delimiter=',')

Loading: ./agModels-spvae_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-spvae_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_drag/models/WeightedEnsemble_L2/model.pkl
